In [1]:
# data_type = {'Dose25', 'Dose50', 'Dose75','Dose100', }
data_type = 'Dose25'

In [2]:
folder_dir = './Medical_images/'

In [3]:
import os
import h5py

female_arr_tr = ['female_1.mat','female_3.mat','female_4.mat','female_5.mat',
             'female_6.mat','female_7.mat','female_8.mat','female_9.mat','female_10.mat',
             'female_11.mat','female_12.mat',]

female_arr_te = ['female_13.mat','female_14.mat','female_15.mat',]

male_arr_tr = ['male_1.mat','male_2.mat','male_3.mat','male_4.mat','male_5.mat',
             'male_6.mat','male_7.mat','male_8.mat','male_9.mat','male_10.mat',
             'male_11.mat',]

male_arr_te = ['male_12.mat','male_13.mat']

name_arr_tr = ['female_1.mat','female_3.mat','female_4.mat','female_5.mat',
             'female_6.mat','female_7.mat','female_8.mat','female_9.mat','female_10.mat',
             'female_11.mat','female_12.mat','male_1.mat','male_2.mat','male_3.mat','male_4.mat','male_5.mat',
             'male_6.mat','male_7.mat','male_8.mat','male_9.mat','male_10.mat',
             'male_11.mat',]

name_arr_te = ['female_13.mat','female_14.mat','female_15.mat','male_12.mat','male_13.mat']

In [7]:
import numpy as np

noisy_data_dir = folder_dir + data_type
tr_noisy_images = []

for name in name_arr_tr:
    data_clean = h5py.File(noisy_data_dir + '/' + name, 'r')['img'][:]
    tr_noisy_images.append(data_clean)
    
noisy = np.array(tr_noisy_images, dtype = np.float32).reshape(22*60,512,512)
print (noisy.shape)

(1320, 512, 512)


In [8]:
clean_data_dir = folder_dir + 'Clean'
tr_clean_images = []

for name in name_arr_tr:
    data_clean = h5py.File(clean_data_dir + '/' + name, 'r')['img_clean'][:]
    tr_clean_images.append(data_clean)
    
clean = np.array(tr_clean_images, dtype = np.float32).reshape(22*60,512,512)
print (clean.shape)

(1320, 512, 512)


In [9]:
import random
def crop_random_patch(clean, noisy, size, num_patches):
    
    rand = random.randrange(1,100000)
    
    clean_patches = image.extract_patches_2d(clean, (size, size), num_patches, rand)
    noisy_patches = image.extract_patches_2d(noisy, (size, size), num_patches, rand)
    
    clean_img = clean_patches.copy()
    noisy_img = noisy_patches.copy()
    
    return clean_img, noisy_img

In [10]:
max_num_patches = 20500
patches_per_image = max_num_patches // (clean.shape[0]) + 1
global_size = 120

print ('max_num_patches : ', max_num_patches)
print ('patches_per_image : ', patches_per_image)

max_num_patches :  20500
patches_per_image :  16


In [11]:
from sklearn.feature_extraction import image
from scipy.ndimage import imread

zi_patch = np.zeros((patches_per_image * (clean.shape[0]),global_size,global_size), dtype = np.float32)
xi_patch = np.zeros((patches_per_image * (noisy.shape[0]),global_size,global_size), dtype = np.float32)

patch_idx = 0

for img_idx in range(noisy.shape[0]):
    
    clean_img = clean[img_idx]
    noisy_img = noisy[img_idx]
    
    xi, zi = crop_random_patch(clean_img, noisy_img, global_size,patches_per_image)
    
    zi_patch[patch_idx:patch_idx+xi.shape[0]] = zi
    xi_patch[patch_idx:patch_idx+xi.shape[0]] = xi

    patch_idx += xi.shape[0]
    
    if patch_idx >= max_num_patches:
        print ('len of zi_patch, xi_patch : ', patch_idx)
        break

len of zi_patch, xi_patch :  20512


In [12]:
import matplotlib.pyplot as plt
plt.figure(figsize=[10,8])
plt.imshow(zi_patch[max_num_patches-1], cmap='gray')
plt.show()

<Figure size 1000x800 with 1 Axes>

In [13]:
file_name = 'G2G_train_120x120_Medical_dataset_'+str(data_type)+'.hdf5'
# file_name = 'G2G_dataset_96x96_RealFM_WF_z_i_lambda42.hdf5'
f = h5py.File(file_name, "w")
f.create_dataset('clean_patches', data=np.array(xi_patch[:max_num_patches]))
f.create_dataset('noisy_patches', data=np.array(zi_patch[:max_num_patches]))
f.close()